# 第十二章 context

Context 是 Go 語言中用於處理請求範圍的值、取消信號和超時的重要機制。它主要用於在多個 goroutine 之間傳遞請求級別的資料，並管理操作的生命週期。

本章將涵蓋：
1. 什麼是 context？
2. 取消
3. 計時器
4. 在你自己的程式中處理 context 取消
5. 值

## 什麼是 context？

Context 是 Go 標準程式庫中的一個重要概念，定義在 `context` 套件中。它提供了一個跨 API 邊界和進程邊界傳遞期限、取消信號和其他請求範圍值的標準方法。

Context 的主要用途包括：
1. **取消傳播**：當一個操作需要停止時，可以通過 context 取消所有相關的子操作
2. **超時控制**：設定操作的最大執行時間
3. **值傳遞**：在請求的生命週期內傳遞請求範圍的資料
4. **截止時間**：設定操作必須完成的具體時間

Context 是一個介面，定義了四個方法：
- `Deadline()` - 返回 context 被取消的時間
- `Done()` - 返回一個 channel，當 context 被取消時會被關閉
- `Err()` - 返回 context 被取消的原因
- `Value(key interface{})` - 返回與 key 關聯的值

In [ ]:
package main

import (
    "context"
    "fmt"
)

/* 基本的 context 介面展示 */
func demonstrateContextInterface() {
    // 建立一個背景 context
    ctx := context.Background()
    
    // 檢查 context 的基本屬性
    deadline, hasDeadline := ctx.Deadline() // 背景 context 沒有截止時間
    fmt.Printf("Has deadline: %t\n", hasDeadline) // 輸出: Has deadline: false
    
    done := ctx.Done() // 背景 context 永遠不會被取消
    fmt.Printf("Done channel is nil: %t\n", done == nil) // 輸出: Done channel is nil: true
    
    err := ctx.Err() // 沒有錯誤
    fmt.Printf("Error: %v\n", err) // 輸出: Error: <nil>
    
    fmt.Printf("Deadline: %v\n", deadline) // 輸出: Deadline: 0001-01-01 00:00:00 +0000 UTC
}

func main() {
    demonstrateContextInterface()
}

In [ ]:
/* context.Background() 和 context.TODO() 的區別 */
func demonstrateRootContexts() {
    // context.Background() 用於主要函式、初始化和測試
    bgCtx := context.Background()
    fmt.Printf("Background context type: %T\n", bgCtx) // 輸出: Background context type: *context.emptyCtx
    
    // context.TODO() 用於當你不確定該使用哪個 context 時
    todoCtx := context.TODO()
    fmt.Printf("TODO context type: %T\n", todoCtx) // 輸出: TODO context type: *context.emptyCtx
    
    // 兩者在功能上是相同的，只是語義不同
    fmt.Printf("Are they equal? %t\n", bgCtx == todoCtx) // 輸出: Are they equal? false (不同的實例)
}

func main() {
    demonstrateRootContexts()
}

In [ ]:
/* Context 的典型使用模式：函式第一個參數 */
func processRequest(ctx context.Context, data string) error {
    // 檢查 context 是否已被取消
    select {
    case <-ctx.Done():
        return ctx.Err() // 返回取消錯誤
    default:
        // 繼續處理
    }
    
    fmt.Printf("Processing: %s\n", data) // 輸出: Processing: hello world
    return nil
}

func handleAPI(ctx context.Context, userID string) {
    // 將 context 傳遞給所有子函式
    err := processRequest(ctx, "hello world")
    if err != nil {
        fmt.Printf("Request failed: %v\n", err)
        return
    }
    fmt.Printf("Request completed for user: %s\n", userID) // 輸出: Request completed for user: 123
}

func main() {
    ctx := context.Background()
    handleAPI(ctx, "123")
}

## 取消

Context 最重要的功能之一是提供取消機制。當一個操作需要停止時，可以通過取消 context 來通知所有使用該 context 的 goroutine 停止工作。

### context.WithCancel

`context.WithCancel` 函式會返回一個新的 context 和一個取消函式。當調用取消函式時，context 的 `Done()` channel 會被關閉，所有監聽該 channel 的 goroutine 都會收到取消信號。

### 取消的傳播

Context 的取消會從父 context 傳播到所有子 context。當父 context 被取消時，所有從它衍生的子 context 也會被取消。

### 取消的最佳實踐

1. 總是調用取消函式以釋放資源
2. 使用 defer 語句確保取消函式被調用
3. 在長時間運行的操作中定期檢查取消信號
4. 不要將取消函式傳遞給其他 goroutine

In [ ]:
package main

import (
    "context"
    "fmt"
    "time"
)

/* 基本的 context.WithCancel 使用 */
func basicCancelExample() {
    // 建立可取消的 context
    ctx, cancel := context.WithCancel(context.Background())
    
    // 使用 defer 確保取消函式被調用
    defer cancel()
    
    go func() {
        select {
        case <-time.After(2 * time.Second):
            fmt.Println("Work completed") // 這行不會執行因為會先被取消
        case <-ctx.Done():
            fmt.Println("Work cancelled:", ctx.Err()) // 輸出: Work cancelled: context canceled
        }
    }()
    
    // 等待 1 秒後取消
    time.Sleep(1 * time.Second)
    cancel()
    time.Sleep(100 * time.Millisecond) // 等待 goroutine 完成
}

func main() {
    basicCancelExample()
}

In [ ]:
/* 取消傳播：父 context 取消會影響子 context */
func cancelPropagationExample() {
    // 父 context
    parentCtx, parentCancel := context.WithCancel(context.Background())
    defer parentCancel()
    
    // 子 context
    childCtx, childCancel := context.WithCancel(parentCtx)
    defer childCancel()
    
    // 孫 context
    grandchildCtx, grandchildCancel := context.WithCancel(childCtx)
    defer grandchildCancel()
    
    // 啟動 goroutine 監聽各個 context
    go func() {
        <-parentCtx.Done()
        fmt.Println("Parent context cancelled") // 輸出: Parent context cancelled
    }()
    
    go func() {
        <-childCtx.Done()
        fmt.Println("Child context cancelled") // 輸出: Child context cancelled
    }()
    
    go func() {
        <-grandchildCtx.Done()
        fmt.Println("Grandchild context cancelled") // 輸出: Grandchild context cancelled
    }()
    
    time.Sleep(100 * time.Millisecond)
    
    // 只取消父 context，所有子 context 也會被取消
    parentCancel()
    
    time.Sleep(100 * time.Millisecond) // 等待所有 goroutine 完成
}

func main() {
    cancelPropagationExample()
}

In [ ]:
/* 在長時間運行的函式中檢查取消信號 */
func longRunningTask(ctx context.Context) error {
    for i := 0; i < 10; i++ {
        // 在每次迭代中檢查是否被取消
        select {
        case <-ctx.Done():
            fmt.Printf("Task cancelled at iteration %d: %v\n", i, ctx.Err())
            return ctx.Err() // 輸出: Task cancelled at iteration 3: context canceled
        default:
            // 繼續工作
        }
        
        fmt.Printf("Working on iteration %d\n", i) // 輸出: Working on iteration 0, 1, 2, 3
        time.Sleep(500 * time.Millisecond)
    }
    
    fmt.Println("Task completed successfully")
    return nil
}

func main() {
    ctx, cancel := context.WithCancel(context.Background())
    
    go func() {
        // 2 秒後取消任務
        time.Sleep(2 * time.Second)
        cancel()
    }()
    
    err := longRunningTask(ctx)
    if err != nil {
        fmt.Printf("Task failed: %v\n", err) // 輸出: Task failed: context canceled
    }
}

## 計時器

Context 提供了兩種與時間相關的功能：超時（timeout）和截止時間（deadline）。這些機制讓你可以限制操作的執行時間，避免程式無限期地等待。

### context.WithTimeout

`context.WithTimeout` 創建一個會在指定持續時間後自動取消的 context。這對於設定操作的最大執行時間很有用。

### context.WithDeadline

`context.WithDeadline` 創建一個會在指定時間點自動取消的 context。適用於需要在特定時間前完成的操作。

### 超時與取消的組合

超時本質上是一種自動取消機制。當時間到達時，context 會自動被取消，並且 `ctx.Err()` 會返回 `context.DeadlineExceeded` 錯誤。

### 最佳實踐

1. 為網路請求設定合理的超時時間
2. 使用超時來避免資源洩漏
3. 根據業務需求選擇適當的超時時間
4. 記住超時是從 context 創建時開始計算的

In [ ]:
package main

import (
    "context"
    "fmt"
    "time"
)

/* context.WithTimeout 基本使用 */
func timeoutExample() {
    // 創建一個 2 秒超時的 context
    ctx, cancel := context.WithTimeout(context.Background(), 2*time.Second)
    defer cancel() // 總是調用 cancel 以釋放資源
    
    // 檢查 context 的截止時間
    deadline, hasDeadline := ctx.Deadline()
    fmt.Printf("Has deadline: %t\n", hasDeadline) // 輸出: Has deadline: true
    fmt.Printf("Deadline: %v\n", deadline) // 輸出當前時間 + 2 秒
    
    select {
    case <-time.After(3 * time.Second):
        fmt.Println("Work completed") // 這行不會執行
    case <-ctx.Done():
        fmt.Println("Context timeout:", ctx.Err()) // 輸出: Context timeout: context deadline exceeded
    }
}

func main() {
    timeoutExample()
}

In [ ]:
/* context.WithDeadline 使用範例 */
func deadlineExample() {
    // 設定一個具體的截止時間（現在 + 1.5 秒）
    deadline := time.Now().Add(1500 * time.Millisecond)
    ctx, cancel := context.WithDeadline(context.Background(), deadline)
    defer cancel()
    
    fmt.Printf("Current time: %v\n", time.Now().Format("15:04:05.000"))
    fmt.Printf("Deadline: %v\n", deadline.Format("15:04:05.000"))
    
    // 模擬一個需要 2 秒的操作
    select {
    case <-time.After(2 * time.Second):
        fmt.Println("Work completed successfully") // 這行不會執行
    case <-ctx.Done():
        fmt.Printf("Deadline exceeded at: %v\n", time.Now().Format("15:04:05.000"))
        fmt.Printf("Error: %v\n", ctx.Err()) // 輸出: Error: context deadline exceeded
    }
}

func main() {
    deadlineExample()
}

In [ ]:
/* 比較手動取消和超時取消 */
func compareTimeoutAndCancel() {
    fmt.Println("=== 超時取消範例 ===")
    
    // 創建 1 秒超時的 context
    timeoutCtx, timeoutCancel := context.WithTimeout(context.Background(), 1*time.Second)
    defer timeoutCancel()
    
    select {
    case <-time.After(2 * time.Second):
        fmt.Println("Timeout work completed") // 不會執行
    case <-timeoutCtx.Done():
        fmt.Printf("Timeout context error: %v\n", timeoutCtx.Err()) // 輸出: context deadline exceeded
    }
    
    fmt.Println("=== 手動取消範例 ===")
    
    // 創建可手動取消的 context
    cancelCtx, cancel := context.WithCancel(context.Background())
    
    go func() {
        time.Sleep(500 * time.Millisecond)
        cancel() // 手動取消
    }()
    
    select {
    case <-time.After(2 * time.Second):
        fmt.Println("Cancel work completed") // 不會執行
    case <-cancelCtx.Done():
        fmt.Printf("Cancel context error: %v\n", cancelCtx.Err()) // 輸出: context canceled
    }
}

func main() {
    compareTimeoutAndCancel()
}

In [ ]:
/* 網路請求的超時控制範例 */
func simulateNetworkRequest(ctx context.Context, url string) error {
    fmt.Printf("Starting request to %s\n", url)
    
    // 模擬網路請求的時間
    requestTime := 3 * time.Second
    
    select {
    case <-time.After(requestTime):
        fmt.Printf("Request to %s completed successfully\n", url)
        return nil
    case <-ctx.Done():
        fmt.Printf("Request to %s cancelled: %v\n", url, ctx.Err())
        return ctx.Err() // 輸出: Request to example.com cancelled: context deadline exceeded
    }
}

func networkTimeoutExample() {
    // 設定 2 秒超時
    ctx, cancel := context.WithTimeout(context.Background(), 2*time.Second)
    defer cancel()
    
    err := simulateNetworkRequest(ctx, "example.com")
    if err != nil {
        fmt.Printf("Network request failed: %v\n", err) // 輸出: Network request failed: context deadline exceeded
    }
}

func main() {
    networkTimeoutExample()
}

## 在你自己的程式中處理 context 取消

當你在自己的應用程式中使用 context 時，需要正確地處理取消信號。這包括在適當的時機檢查 context 狀態、清理資源，以及向呼叫者傳播取消狀態。

### 設計接受 context 的 API

在設計函式時，應該將 context 作為第一個參數。這已成為 Go 社群的慣例，所有標準程式庫的函式都遵循這個模式。

### 定期檢查取消狀態

在長時間運行的操作中，應該定期檢查 context 是否被取消。這可以通過 select 語句或直接檢查 `ctx.Done()` channel 來實現。

### 資源清理

當接收到取消信號時，應該適當地清理已分配的資源，如關閉檔案、釋放記憶體、關閉網路連接等。

### 快速返回

收到取消信號後應該儘快返回，不要繼續執行不必要的操作。

In [ ]:
package main

import (
    "context"
    "fmt"
    "time"
)

/* 設計接受 context 的 API 函式 */
func processData(ctx context.Context, data []int) ([]int, error) {
    result := make([]int, 0, len(data))
    
    for i, v := range data {
        // 在每次迭代中檢查取消狀態
        select {
        case <-ctx.Done():
            fmt.Printf("Processing cancelled at index %d\n", i) // 可能輸出: Processing cancelled at index 3
            return nil, ctx.Err()
        default:
            // 繼續處理
        }
        
        // 模擬處理時間
        time.Sleep(200 * time.Millisecond)
        
        // 處理數據（這裡只是簡單的平方）
        processed := v * v
        result = append(result, processed)
        fmt.Printf("Processed %d -> %d\n", v, processed)
    }
    
    fmt.Println("All data processed successfully")
    return result, nil
}

func main() {
    data := []int{1, 2, 3, 4, 5, 6, 7, 8}
    
    // 設定 1 秒超時
    ctx, cancel := context.WithTimeout(context.Background(), 1*time.Second)
    defer cancel()
    
    result, err := processData(ctx, data)
    if err != nil {
        fmt.Printf("Processing failed: %v\n", err) // 輸出: Processing failed: context deadline exceeded
    } else {
        fmt.Printf("Result: %v\n", result)
    }
}

In [ ]:
/* 正確的資源清理與 context 取消處理 */
type Resource struct {
    name string
}

func (r *Resource) Open() {
    fmt.Printf("Opening resource: %s\n", r.name)
}

func (r *Resource) Close() {
    fmt.Printf("Closing resource: %s\n", r.name)
}

func (r *Resource) Process() {
    fmt.Printf("Processing with resource: %s\n", r.name)
    time.Sleep(500 * time.Millisecond) // 模擬處理時間
}

func handleWithResources(ctx context.Context, resourceNames []string) error {
    var resources []*Resource
    
    // 創建和開啟資源
    for _, name := range resourceNames {
        resource := &Resource{name: name}
        resource.Open()
        resources = append(resources, resource)
    }
    
    // 確保資源會被清理
    defer func() {
        fmt.Println("Starting resource cleanup...")
        for i := len(resources) - 1; i >= 0; i-- {
            resources[i].Close() // 輸出: Closing resource: resource2, resource1
        }
        fmt.Println("Resource cleanup completed")
    }()
    
    // 處理每個資源
    for i, resource := range resources {
        select {
        case <-ctx.Done():
            fmt.Printf("Context cancelled while processing resource %d\n", i)
            return ctx.Err() // 輸出: context deadline exceeded，然後觸發 defer 清理
        default:
            resource.Process() // 輸出: Processing with resource: resource1
        }
    }
    
    fmt.Println("All resources processed successfully")
    return nil
}

func main() {
    ctx, cancel := context.WithTimeout(context.Background(), 800*time.Millisecond)
    defer cancel()
    
    resources := []string{"resource1", "resource2"}
    err := handleWithResources(ctx, resources)
    if err != nil {
        fmt.Printf("Handler failed: %v\n", err) // 輸出: Handler failed: context deadline exceeded
    }
}

In [ ]:
/* Context 取消的傳播與 Goroutine 管理 */
func workerWithContext(ctx context.Context, workerID int, workCh <-chan int, resultCh chan<- int) {
    fmt.Printf("Worker %d started\n", workerID)
    defer fmt.Printf("Worker %d stopped\n", workerID)
    
    for {
        select {
        case work, ok := <-workCh:
            if !ok {
                return // channel 已關閉
            }
            
            // 檢查是否應該取消
            select {
            case <-ctx.Done():
                fmt.Printf("Worker %d cancelled while processing %d\n", workerID, work)
                return
            default:
            }
            
            // 處理工作
            fmt.Printf("Worker %d processing %d\n", workerID, work)
            time.Sleep(200 * time.Millisecond) // 模擬工作
            
            // 再次檢查取消（因為處理可能需要時間）
            select {
            case <-ctx.Done():
                fmt.Printf("Worker %d cancelled after processing %d\n", workerID, work)
                return
            case resultCh <- work*2:
                fmt.Printf("Worker %d completed %d -> %d\n", workerID, work, work*2)
            }
            
        case <-ctx.Done():
            fmt.Printf("Worker %d received cancellation signal\n", workerID)
            return
        }
    }
}

func coordinateWorkers() {
    ctx, cancel := context.WithTimeout(context.Background(), 1*time.Second)
    defer cancel()
    
    workCh := make(chan int, 5)
    resultCh := make(chan int, 5)
    
    // 啟動 workers
    for i := 1; i <= 3; i++ {
        go workerWithContext(ctx, i, workCh, resultCh)
    }
    
    // 發送工作
    work := []int{10, 20, 30, 40, 50}
    go func() {
        defer close(workCh)
        for _, w := range work {
            select {
            case workCh <- w:
                fmt.Printf("Sent work: %d\n", w)
            case <-ctx.Done():
                fmt.Println("Stopped sending work due to cancellation")
                return
            }
        }
    }()
    
    // 收集結果
    for i := 0; i < len(work); i++ {
        select {
        case result := <-resultCh:
            fmt.Printf("Received result: %d\n", result)
        case <-ctx.Done():
            fmt.Printf("Stopped collecting results: %v\n", ctx.Err())
            return
        }
    }
}

func main() {
    coordinateWorkers()
}

## 值

Context 除了提供取消機制外，還可以攜帶請求範圍的值。這些值可以在整個請求的生命週期內在不同的函式之間傳遞，而不需要顯式地將它們作為參數傳遞。

### context.WithValue

`context.WithValue` 函式創建一個攜帶鍵值對的新 context。這個值可以通過 `ctx.Value(key)` 方法獲取。

### 使用 Context 值的最佳實踐

1. **只用於請求範圍的資料**：不要用 context 傳遞可選參數
2. **使用自定義型別作為鍵**：避免鍵衝突
3. **值應該是不可變的**：避免意外的副作用
4. **不要過度使用**：context 值主要用於跨越API邊界的資料
5. **提供存取器函式**：為取得 context 值提供專門的函式

### 常見的 Context 值用例

- 請求 ID 或追蹤 ID
- 使用者認證資訊
- 請求的開始時間
- 地區設定資訊（如語言、時區）

### 注意事項

Context 值查找是通過遍歷 context 鏈來實現的，所以過深的嵌套可能會影響性能。另外，context 值不應該用來傳遞可選的函式參數。

In [ ]:
package main

import (
    "context"
    "fmt"
)

/* 基本的 context.WithValue 使用 */
func basicValueExample() {
    // 創建攜帶值的 context
    ctx := context.WithValue(context.Background(), "userID", "user123")
    
    // 從 context 中獲取值
    userID := ctx.Value("userID")
    if userID != nil {
        fmt.Printf("User ID: %v\n", userID) // 輸出: User ID: user123
    }
    
    // 獲取不存在的值
    nonExistent := ctx.Value("nonexistent")
    fmt.Printf("Non-existent value: %v\n", nonExistent) // 輸出: Non-existent value: <nil>
}

func main() {
    basicValueExample()
}

In [ ]:
/* 使用自定義型別作為鍵的最佳實踐 */
type contextKey string

const (
    UserIDKey    contextKey = "userID"
    RequestIDKey contextKey = "requestID"
    TraceIDKey   contextKey = "traceID"
)

// 提供存取器函式
func GetUserID(ctx context.Context) (string, bool) {
    userID, ok := ctx.Value(UserIDKey).(string)
    return userID, ok
}

func GetRequestID(ctx context.Context) (string, bool) {
    requestID, ok := ctx.Value(RequestIDKey).(string)
    return requestID, ok
}

func WithUserID(ctx context.Context, userID string) context.Context {
    return context.WithValue(ctx, UserIDKey, userID)
}

func WithRequestID(ctx context.Context, requestID string) context.Context {
    return context.WithValue(ctx, RequestIDKey, requestID)
}

func bestPracticeExample() {
    // 建立攜帶多個值的 context
    ctx := context.Background()
    ctx = WithUserID(ctx, "user456")
    ctx = WithRequestID(ctx, "req-789")
    
    // 使用存取器函式獲取值
    if userID, ok := GetUserID(ctx); ok {
        fmt.Printf("Found User ID: %s\n", userID) // 輸出: Found User ID: user456
    }
    
    if requestID, ok := GetRequestID(ctx); ok {
        fmt.Printf("Found Request ID: %s\n", requestID) // 輸出: Found Request ID: req-789
    }
    
    // 檢查不存在的值
    if traceID, ok := ctx.Value(TraceIDKey).(string); ok {
        fmt.Printf("Found Trace ID: %s\n", traceID)
    } else {
        fmt.Println("Trace ID not found") // 輸出: Trace ID not found
    }
}

func main() {
    bestPracticeExample()
}

In [ ]:
/* 實際應用範例：Web 請求處理 */
type User struct {
    ID   string
    Name string
    Role string
}

type Logger struct {
    requestID string
}

func (l *Logger) Log(msg string) {
    fmt.Printf("[%s] %s\n", l.requestID, msg)
}

func NewLogger(ctx context.Context) *Logger {
    requestID, _ := GetRequestID(ctx)
    return &Logger{requestID: requestID}
}

// 模擬認證函式
func authenticate(ctx context.Context) context.Context {
    // 模擬從請求中獲取使用者資訊
    user := &User{
        ID:   "user123",
        Name: "Alice",
        Role: "admin",
    }
    
    // 將使用者資訊添加到 context
    return context.WithValue(ctx, "user", user)
}

// 業務邏輯函式
func processRequest(ctx context.Context, data string) error {
    logger := NewLogger(ctx)
    logger.Log("Starting request processing") // 輸出: [req-456] Starting request processing
    
    // 獲取使用者資訊
    user, ok := ctx.Value("user").(*User)
    if !ok {
        logger.Log("No user found in context")
        return fmt.Errorf("unauthenticated")
    }
    
    logger.Log(fmt.Sprintf("Processing request for user: %s (%s)", user.Name, user.Role))
    // 輸出: [req-456] Processing request for user: Alice (admin)
    
    // 根據使用者角色處理請求
    if user.Role == "admin" {
        logger.Log("Admin access granted - processing data: " + data)
        // 輸出: [req-456] Admin access granted - processing data: important data
    } else {
        logger.Log("Regular user access - limited processing")
    }
    
    logger.Log("Request processing completed") // 輸出: [req-456] Request processing completed
    return nil
}

func webRequestExample() {
    // 模擬 Web 請求處理流程
    ctx := context.Background()
    
    // 添加請求 ID
    ctx = WithRequestID(ctx, "req-456")
    
    // 認證階段
    ctx = authenticate(ctx)
    
    // 處理請求
    err := processRequest(ctx, "important data")
    if err != nil {
        fmt.Printf("Request failed: %v\n", err)
    }
}

func main() {
    webRequestExample()
}

In [ ]:
/* Context 值的繼承與組合：結合取消、超時與值 */
func comprehensiveContextExample() {
    // 建立基礎 context 攜帶請求資訊
    ctx := context.Background()
    ctx = WithRequestID(ctx, "req-comprehensive-123")
    ctx = WithUserID(ctx, "user999")
    
    // 添加超時
    ctx, cancel := context.WithTimeout(ctx, 2*time.Second)
    defer cancel()
    
    fmt.Println("=== Context 全功能展示 ===")
    
    // 啟動一個 goroutine 處理任務
    go func() {
        processWithFullContext(ctx, "sensitive operation")
    }()
    
    // 主 goroutine 等待一會後手動取消
    time.Sleep(500 * time.Millisecond)
    fmt.Println("Main: Manually cancelling context...")
    cancel()
    
    // 等待 goroutine 完成
    time.Sleep(200 * time.Millisecond)
}

func processWithFullContext(ctx context.Context, operation string) {
    logger := NewLogger(ctx)
    
    logger.Log("Operation started: " + operation) // 輸出: [req-comprehensive-123] Operation started: sensitive operation
    
    // 檢查使用者 ID
    if userID, ok := GetUserID(ctx); ok {
        logger.Log("Processing for user: " + userID) // 輸出: [req-comprehensive-123] Processing for user: user999
    }
    
    // 模擬長時間運行的操作
    for i := 0; i < 5; i++ {
        select {
        case <-ctx.Done():
            logger.Log(fmt.Sprintf("Operation cancelled at step %d: %v", i, ctx.Err()))
            // 輸出: [req-comprehensive-123] Operation cancelled at step 1: context canceled
            return
        default:
            logger.Log(fmt.Sprintf("Step %d completed", i))
            time.Sleep(300 * time.Millisecond)
        }
    }
    
    logger.Log("Operation completed successfully")
}

func main() {
    comprehensiveContextExample()
}

## 總結

Context 是 Go 語言中管理請求生命週期的核心機制，它提供了三個主要功能：

### 1. 取消機制
- 使用 `context.WithCancel` 創建可取消的 context
- 取消會從父 context 傳播到所有子 context
- 應該在長時間運行的操作中定期檢查 `ctx.Done()`
- 總是使用 `defer cancel()` 確保資源被釋放

### 2. 超時控制
- `context.WithTimeout` 用於設定相對超時時間
- `context.WithDeadline` 用於設定絕對截止時間
- 超時錯誤為 `context.DeadlineExceeded`
- 適用於網路請求、I/O 操作等有時間限制的場景

### 3. 值傳遞
- 使用 `context.WithValue` 攜帶請求範圍的資料
- 只用於跨 API 邊界的元資料，不要用於傳遞可選參數
- 使用自定義型別作為鍵避免衝突
- 提供存取器函式封裝值的讀取邏輯

### Context 使用的最佳實踐

1. **函式設計**：將 context 作為函式的第一個參數
2. **資源管理**：使用 defer 確保取消函式被調用
3. **錯誤處理**：正確處理 context 的錯誤類型
4. **性能考量**：避免過度深層的 context 嵌套
5. **測試**：在測試中使用 context 來控制執行流程

Context 是 Go 並發編程的重要工具，掌握它的正確用法對於寫出健壯、可控制的 Go 應用程式至關重要。